# Save and Load networks

Quick example on network creation, saving into database and loading it later

### Test data generation

In [1]:
import json
from dfg_rating.db.postgres import PostgreSQLDriver
from dfg_rating.logic.controller import Controller
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.forecast.base_forecast import SimpleForecast
from dfg_rating.model.network.base_network import BaseNetwork

The main controller has functions for managing the db.

In [2]:
main_controller = Controller()

Creating some test data

In [ ]:
test_network: BaseNetwork = factory.new_network(
    'multiple-round-robin', 
    teams=18, 
    days_between_rounds=3,
    seasons=4,
    league_teams=18,
    league_promotion=0,
)
test_network.add_forecast(
    forecast=SimpleForecast(outcomes=['home', 'draw', 'away'], probs=[0.4523, 0.2975, 0.2502]),
    forecast_name='simple_forecast',
    base_ranking='true_rating'
)
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=-0.1)
)
test_network.add_odds("simple", bookmaker, "true_forecast")
betting = FixedBetting(1000)
test_network.add_bets(
    "true_bettor",
    "simple",
    betting,
    "simple_forecast"
)
rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name="simple_forecast")
betting_returns = BettingReturnsEvaluator(
    outcomes=['home', 'draw', 'away'], bookmaker_name="simple", player_name="true_bettor", true_model="true_forecast"
)
test_network.add_evaluation(
    evaluators_list=[(rps, "RPS_simple"), (betting_returns, "betting_returns_simple")]
)

Adding the network to the controller as *test_network*

In [ ]:
main_controller.networks["test"] = test_network

Saving network

In [ ]:
main_controller.save_network("test")

In [3]:
main_controller.load_network_from_sql(network_name="test", new_network_name="read_test")

('PostgreSQL 12.8 (Ubuntu 12.8-0ubuntu0.20.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
dfg_rating database exists. Loading database
Total of 1224 matches.
Total of 2448 forecasts.
Total of 1278 ratings.
Total of 1224 odds.
Total of 1224 bets.
Total of 2448 metrics.
['0', '9', '1', '10', '2', '11', '3', '12', '4', '13', '5', '14', '6', '15', '7', '16', '8', '17']


(1, 'Network loaded correctly')

In [5]:
main_controller.networks.keys()

dict_keys(['read_test'])

In [6]:
import dfg_rating.viz.jupyter_widgets as DFGWidgets

In [7]:
#test_app = DFGWidgets.NetworkExplorer(network=main_controller.networks['test'])
read_app = DFGWidgets.NetworkExplorer(network=main_controller.networks['read_test'])

In [10]:
read_app.run('inline', port=8051)